In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!nvidia-smi

Tue May  4 13:30:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -U git+https://github.com/aarushvarma75/efficientnet.git
!pip install keras_flops
!pip install -q kaggle

  Cloning https://github.com/aarushvarma75/efficientnet.git to /tmp/pip-req-build-6dnqyx18
  Running command git clone -q https://github.com/aarushvarma75/efficientnet.git /tmp/pip-req-build-6dnqyx18
     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for efficientnet: filename=efficientnet-1.1.1-cp37-none-any.whl size=18478 sha256=2c14eaf48bcb6bde356b8f97e3a72622c0d3531d70a7cde8889b3b752eb811ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-j52u82lg/wheels/c2/5c/59/80cee904f471431f5565590f0139d5c89d66b92388e2c1b0f8
Successfully built efficientnet


In [4]:
from google.colab import files
files.upload()      

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aarushverma","key":"a556b53644acf230d4623a725d843ac2"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d miljan/stanford-dogs-dataset-traintest 

100% 393M/393M [00:07<00:00, 50.9MB/s]



In [7]:
! mkdir train_test
! unzip stanford-dogs-dataset-traintest.zip -d train_test

Streaming output truncated to the last 5000 lines.
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1066.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1140.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1200.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1270.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1299.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1318.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1409.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_158.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1615.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_spaniel/n02102973_1622.jpg  
  inflating: train_test/cropped/train/n02102973-Irish_water_

In [8]:
import numpy as np
import multiprocessing
from copy import deepcopy

In [9]:
def model_train_compile(alpha,beta,gamma,cost): 
    import sys
    import tensorflow as tf
    import multiprocessing
    import os
    from keras_flops import get_flops
    sys.path.append('..')  
    sys.stdout = open("std.out", "w")
    sys.stderr = open("error.err", "w")
    
    from content.gdrive.MyDrive.efficientnet.tfkeras import EfficientNetB0
    batch_size = 32
    IMG_SIZE = int(224*gamma)
    img_height = int(224*gamma)
    img_width = int(224*gamma)

    ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/train_test/cropped/train',
    labels='inferred',
      label_mode='categorical', 
    image_size=(img_height, img_width),
    batch_size=batch_size) 

    tf.keras.backend.clear_session()
    model = EfficientNetB0(alpha, beta, gamma, weights = None)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2) 
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    hist = model.fit(ds_train.take(30), epochs=80, verbose=1)
    loss = hist.history['loss'][-1]
    print(loss)
    flops = get_flops(model, batch_size=1)
    print(f"FLOPS: {flops / 10 ** 9:.03} G")
    target_flops = 4e+8
    print(flops/target_flops)
    fness = 1/((loss)*((flops/target_flops)**0.07))
    cost.value = fness

In [10]:
def fitness(pos):
    alpha, beta, gamma = pos
    cost = multiprocessing.Value('f')
    p = multiprocessing.Process(target=model_train_compile, args=(alpha,beta,gamma,cost))
    p.start()
    p.join()
    fness = cost.value
    penalty = (alpha*(beta**2)*(gamma**2) - 2)**2
    return fness - 50*penalty

In [11]:
class Particle():
    ''' A single partice, part of a swarm for PSO '''
    
    def __init__(self, x, y, z, vx, vy, vz): # Task 1
        ''' initialize the particle'''
        # YOUR CODE HERE
        self.pos = np.array([x,y,z])
        self.vel = np.array([vx,vy,vz])
        self.best_pos = self.pos
        #raise NotImplementedError()
    
    def __str__(self):
        ''' Controls how a particle is printed '''
        return f'Particle: pos: {self.pos}, velocity: {self.vel} fitness: {fitness(self.pos)}'
    
    def __repr__(self):
        ''' Controls how a particle is printed as a part of a list'''
        return f'Particle: pos: {self.pos}, velocity: {self.vel} fitness: {fitness(self.pos)}'        

In [12]:
def initialize_swarm(hyper_params): # task 2
    '''
    Initialize a swarm of particles using the Particle class
    '''
    particles = []
    fitnesses = []
    p_lo, p_hi = hyper_params['pos_range']
    v_lo, v_hi = hyper_params['vel_range']

    for i in range(hyper_params['pop_size']):
        print(str(i) + '/' + str(hyper_params['pop_size']))
        # Create a particle with random position and random velocity and add it to the particles array
        # YOUR CODE HERE
        x = np.clip(np.random.random()*(p_hi-p_lo)+np.abs(p_lo), p_lo, p_hi)
        y = np.clip(np.random.random()*(p_hi-p_lo)+np.abs(p_lo), p_lo, p_hi)
        z = np.clip(np.random.random()*(p_hi-p_lo)+np.abs(p_lo), p_lo, p_hi)
        vx = np.random.random()*(v_hi-v_lo)-np.abs(v_lo)
        vy = np.random.random()*(v_hi-v_lo)-np.abs(v_lo)
        vz = np.random.random()*(v_hi-v_lo)-np.abs(v_lo)
        particle = Particle(x,y,z,vx,vy,vz)
        fitness_particle = fitness(particle.pos)
        particles.append([particle, fitness_particle])
        #raise NotImplementedError()
        
    fitnesses = [p[1] for p in particles]
    best_fitness = max(fitnesses)
    ix = fitnesses.index(best_fitness)
    best_particle = particles[ix][0]
    best_pos = [best_particle.pos, best_fitness]

    return {
        'best_pos': best_pos,
        'population': particles,
    }

In [13]:
def update_velocity(hyper_params, swarm, particle ): # task 3  
    '''
    Update the given particle's velocity
    '''
    w, c1, c2 = hyper_params['w'], hyper_params['c1'], hyper_params['c2']
    # YOUR CODE HERE
    particle.vel = ((w*particle.vel) +(c1*np.random.rand()*(particle.best_pos - particle.pos))+(c2*np.random.rand()*(swarm['best_pos'][0] - particle.pos)))/(w+c1+c2)
    particle.vel = np.vectorize(lambda z: round(z,3))(particle.vel)
    #raise NotImplementedError()

In [14]:
def update_position(hyper_params, particle_and_fitness): # task 4
    '''
    Update the given particle's position
    '''
    
    # YOUR CODE HERE
    particle, particle_best_fitness = particle_and_fitness
    p_lo, p_hi = hyper_params['pos_range']
    print('Pos before update: ' + str(particle.pos))
    particle.pos = particle.pos + particle.vel
    particle.pos = np.clip(particle.pos, p_lo, p_hi)
    particle.pos = np.vectorize(lambda z: round(z,3))(particle.pos)
    print('Pos after update: ' + str(particle.pos))
    current_fitness = fitness(particle.pos)
    print('Current fitness and particle best fitness:' + str(current_fitness) + ' / ' + str(particle_and_fitness[1]))
    if(current_fitness > particle_and_fitness[1]):
        particle.best_pos = particle.pos
        particle_and_fitness[1] = current_fitness 
    #raise NotImplementedError()

In [15]:
def PSO(hyper_params):
    '''
    Implementing Particle Swarm Optimization 
    '''
    history = []

    swarm = initialize_swarm(hyper_params)
    count = 0
    for i in range(hyper_params['iterations']):
        print('Best Fitness before iteration ' + str(i + 1) + ':' + str(swarm['best_pos'][1]) + ' at pos:' + str(swarm['best_pos'][0]))
        for j in range(hyper_params['pop_size']):
        # YOUR CODE HERE
            update_velocity(hyper_params,swarm,swarm['population'][j][0])
            update_position(hyper_params,swarm['population'][j])
            count = count + 1
            print(str(count) + '/' + str(hyper_params['iterations']*hyper_params['pop_size']) + '\n')
        fitnesses = [p[1] for p in swarm['population']]
        best_fitness = max(fitnesses)
        print('Best Fitness before iteration ' + str(i + 1) + ':' + str(swarm['best_pos'][1]) + ' at pos:' + str(swarm['best_pos'][0]))
        print('Best Fitness after iteration: ' + str(best_fitness) + '\n')
        ix = fitnesses.index(best_fitness)
        best_particle = swarm['population'][ix][0]
        best_fitness_before_update = swarm['best_pos'][1]
        if(best_fitness > best_fitness_before_update):
            swarm['best_pos'][0] = best_particle.pos
            swarm['best_pos'][1] = best_fitness
        
        history.append(deepcopy(swarm)) # Do not modify, this is used for visualization
    #raise NotImplementedError()
        
    
        
    return swarm['best_pos'], history

In [16]:
hyper_params = {
    'w': 1, # inertia 
    'c1': 0.5, # local
    'c2': 0.6, # global
    'pos_range': (1,1.2), # range in which a PSO particle can move
    'vel_range': (-0.05,0.05), # velocity range for a PSO particle
    'pop_size': 3, # Number of particles in the swarm
    'iterations': 5, # Number of iterations
}

optimum, hist = PSO(hyper_params)
print(f'PSO found the optimum at {optimum[0]} with a fitness of {optimum[1]}')

0/3


KeyboardInterrupt: ignored